Import des librairies nécessaires: 

In [42]:
import pandas as pd
import os

Intégration et nettoyage des données: 

In [87]:
## Définition de fonction d'intégration
def charger(path, sep=";"):
    df = pd.read_csv(path, sep=sep)
    return df

def nettoyer(df):
    df = df.drop_duplicates()
    df = df.fillna("NA")
    return df

def sauvegarder(df, dossier_sortie):
    os.makedirs(dossier_sortie, exist_ok=True)
    nom_fichier = str(df)
    chemin = os.path.join(dossier_sortie, nom_fichier)
    df.to_csv(path, index=False)


In [77]:
## Création des tables de correspondances

def tables_correspondances(meta):
    tables = {}
    variables = meta['COD_VAR'].unique()

    for var in variables:
        sous_table = meta[meta['COD_VAR'] == var]
        correspondance = sous_table[['COD_MOD', 'LIB_MOD']].drop_duplicates()
        tables[var] = correspondance
    return tables

def sauvegarder_tables(tables, dossier_sortie):
    os.makedirs(dossier_sortie, exist_ok=True)

    for var, table in tables.items():
        nom_fichier = f"{var}_correspondance.csv"
        chemin = os.path.join(dossier_sortie, nom_fichier)
        table.to_csv(chemin, index=False)
        print(f"Correspondances '{var}' enregistrée : {chemin}")

In [45]:
## Chargement des données
meta = nettoyer(charger("../DS_DEVDUR_metadata.csv",";"))
data = nettoyer(charger("../DS_DEVDUR_data.csv",";"))

In [46]:
data.head(5)

,ODD,INDICATEUR_DEVDUR,UNIT_MEASURE,UNIT_MULT,OBS_STATUS,GEO,GEO_OBJECT,DEVDUR_COMPOSITE,SEX,AGE,EMPSTA,PCS,TYPE_INDICATEUR_DEVDUR,TIME_PERIOD,OBS_VALUE
0,ODD1,1.i1a,PT,0,A,FM,FRANCE,NIVIEMED_S60,_T,_T,11,_T,I_ODD,2015,6.5
1,ODD3,3.i2,_Z,0,E,F,FRANCE,CONTAMIN_MOD4,_T,_T,_Z,_Z,I_ODD,2021,255.0
2,ODD2,2.i2f,PT,0,L,F,FRANCE,_Z,_T,_Z,_Z,1_2,I_ODD,2015,NA
3,ODD1,1.i1a,PT,0,A,FM,FRANCE,NIVIEMED_S60,_T,Y40T49,_T,_T,I_ODD,2016,13.5
4,ODD11,11.i1,PT,0,L,FM,FRANCE,_Z,_T,_T,_Z,_Z,I_ODD,2015,NA


In [69]:
meta.head(5) 

,COD_VAR,LIB_VAR,COD_MOD,LIB_MOD
0,AGE,Âge,Y18T75,De 18 à 75 ans
1,AGE,Âge,Y18T64,De 18 à 64 ans
2,AGE,Âge,_Z,Non applicable
3,AGE,Âge,Y15,15 ans
4,AGE,Âge,Y_LT14,Moins de 14 ans


In [86]:
## Sauvegarde des tables pour exploitation:
data = tables_correspondances(meta), "../DATA")

NameError: name 'path' is not defined

Transformation des données: 

In [71]:
correspondances = sauvegarder_tables(tables_correspondances(meta), "../DATA")

Correspondances 'AGE' enregistrée : ../DATA/AGE_correspondance.csv
Correspondances 'DEVDUR_COMPOSITE' enregistrée : ../DATA/DEVDUR_COMPOSITE_correspondance.csv
Correspondances 'EMPSTA' enregistrée : ../DATA/EMPSTA_correspondance.csv
Correspondances 'INDICATEUR_DEVDUR' enregistrée : ../DATA/INDICATEUR_DEVDUR_correspondance.csv
Correspondances 'OBS_STATUS' enregistrée : ../DATA/OBS_STATUS_correspondance.csv
Correspondances 'ODD' enregistrée : ../DATA/ODD_correspondance.csv
Correspondances 'PCS' enregistrée : ../DATA/PCS_correspondance.csv
Correspondances 'SEX' enregistrée : ../DATA/SEX_correspondance.csv
Correspondances 'TIME_PERIOD' enregistrée : ../DATA/TIME_PERIOD_correspondance.csv
Correspondances 'TYPE_INDICATEUR_DEVDUR' enregistrée : ../DATA/TYPE_INDICATEUR_DEVDUR_correspondance.csv
Correspondances 'UNIT_MEASURE' enregistrée : ../DATA/UNIT_MEASURE_correspondance.csv
Correspondances 'UNIT_MULT' enregistrée : ../DATA/UNIT_MULT_correspondance.csv
Correspondances 'GEO' enregistrée : ..